In [ ]:
import json,gzip,glob
import csv
import pandas as pd
import re
import mwxml
import bz2
import itertools
import requests

In [ ]:
#Setting Current Time (Nov.20th)
date = '20201120'

In [ ]:
#Language & Seeds
lang = ['en','ko','ar','de','es','fr','it','ja','pt','zh','ru']
seeds = ['Q81068910','Q82069695','Q84263196']

In [ ]:
#Get seeds from pagelink.sql file
!grep -E "Q81068910|Q82069695|Q84263196" wikidatawiki-20201120-pagelinks.sql > COVID_item_zgrepmatch.txt

In [ ]:
#Get wikidata page ids which linked to COVID-19 seed items
wikidataPagesIds = []
seeds = ['Q81068910','Q82069695','Q84263196']
f = open('COVID_item_zgrepmatch.txt')
for l  in f:
    for s in seeds:
        wikidataPagesIds.extend(re.findall("\(([0-9]+),0,\'%s\'" % s,l))

In [ ]:
with open('wikidata_QIDs.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(COVID_related_QIDs)

In [ ]:
#MySQL Connection
import pymysql

## Get Article QIDs

In [ ]:
#Get related QIDs from wikidata page ids (dump filename : wikidata_page.sql.gz)
conn = pymysql.connect(host='node200',user='jaehyeon',password='', db='testdb',charset='utf8')
cursor = conn.cursor(pymysql.cursors.DictCursor)

sql = "SELECT page_id,page_title FROM page WHERE page_id IN ({})".format(','.join(wikidataPagesIds))
cursor.execute(sql)
COVID_related_QIDs = [page['page_title'].decode() for page in cursor.fetchall() if page['page_title'].decode('utf8')[0]=='Q'] 
conn.close()

## Get Wikipedia PageIDs

In [ ]:
#Retrieve pageid for each languages (dump filename : wikipedia_page_prop.sql.gz)
conn = pymysql.connect(host='node200',user='jaehyeon',password='', db='testdb',charset='utf8')
cursor = conn.cursor(pymysql.cursors.DictCursor)

wikidata_dict = {}
sql_QIDs = ','.join("'{0}'".format(QID) for QID in COVID_related_QIDs)
for lan in lang:
    sql = f"SELECT pp_page,pp_propname,pp_value FROM {lan}_page_props WHERE pp_value IN ({sql_QIDs}) AND pp_propname = 'wikibase_item'"
    cursor.execute(sql)
    id_dict = {}
    for page in cursor.fetchall():
        id_dict[page['pp_value'].decode()] = {'QID' : page['pp_value'].decode(),
                           'PageID' : page['pp_page'],
                           'Language' : lan+"wiki"} 
    wikidata_dict[lan] = id_dict
conn.close()

## Get Wikipedia Title 

In [ ]:
n = 50
for l in lang:
    print(l)
    qids = list(wikidata_dict[l].keys())
    seg_qid_list = [qids[i * n:(i + 1) * n] for i in range((len(qids) + n - 1) // n )] 
    for ids in seg_qid_list:
        url_ids = '|'.join(ids)
        url = "https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&props=sitelinks&ids={}&sitefilter={}wiki".format(url_ids,l)
        entities = requests.get(url).json().get('entities')
        print(entitites)
        for k,v in entities.items():
            item = entities.get(k)
            if item:
                sitelink = item.get('sitelinks')
                if sitelink:
                    title = sitelink.get(l+'wiki').get('title')
                    if title:
                        wikidata_dict[l][k]['Title'] = title

## Set Category

Region & Human Category pages are selected from Wikidata "instance of" label. If label is "human" then category is "Human". Else if label is "outbreak", category is "Region". Label data can be collected from wikidata query.

In [ ]:
#Region Categories
region_qids = []
f = open('region.csv', 'r', encoding='utf-8')
rdr = csv.reader(f)
for line in rdr:
    region_qids.append(line[0].split('/')[-1])
f.close()    
region_qids = region_qids[1:]

In [ ]:
#Human Categories
human_qids = []
f = open('human.csv', 'r', encoding='utf-8')
rdr = csv.reader(f)
for line in rdr:
    human_qids.append(line[0].split('/')[-1])
f.close()    
region_qids = region_qids[1:]

In [ ]:
for l in lang:
    for k, v in wikidata_dict[l].items():
        if k in ['Q81068910','Q82069695','Q84263196']:
            v['category'] = 'Bio-Med'
        elif k in region_qids:
            v['category'] = 'Region'
        elif k in human_qids:
            v['category'] = 'Human'
        else:
            v['category'] = 'Others'
            

## CSV Save

In [ ]:
for l in lang:
    name = "wikipedia_list_cleaned_1120_{}.csv".format(l)
    with open(name,'w') as f:
        writer = csv.writer(f)
        writer.writerow(('QID','page id','wiki_db','item','category'))
        for key, value in wikidata_dict[l].items():
            writer.writerow(list(value.values()))

In [ ]:
for l in lang:
    print(l,len(wikidata_dict[l].keys()))

In [ ]:
with open('1120_data.json','w') as f:
    json.dump(wikidata_dict,f)

## Get Common List

In [ ]:
qids = [list(wikidata_dict[l].keys()) for l in lang]
common_list = list(set(qids[0]).intersection(*qids))

name = "wikipedia_common_list_1120.csv"
with open(name,'w') as f:
    writer = csv.writer(f)
    writer.writerow(common_list)
